In [69]:
pip install pandas openai together anthropic


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [70]:
import openai
import re
import random
import anthropic
from collections import Counter



In [71]:
import os
import requests

client = anthropic.Anthropic()
import os


In [72]:
import pandas as pd
import glob
import os

path = "../data/iSarcasmEval.csv"
df = pd.read_csv(path);

answers = df['sarcastic']
statements = df['tweet']


In [73]:
outputs =[]

In [74]:
from anthropic import Anthropic

def getResultNoPrompt(input, answer):
    anthropic = Anthropic()

    prompt = f"\"{input}\"\n Determine whether this statement is sarcastic with YES or NO."

    response = anthropic.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=3000,
        temperature=1.0
    )

    output = response.content[0].text.strip()
    isSarcastic = True if "yes" in output.lower() else False
    return isSarcastic

In [76]:
from anthropic import Anthropic

def getResultWPrompt(sample_input):
   anthropic = Anthropic()
   judgeInput = ""
   output = ""

   # System prompt for sarcasm justification
   sarc_system_prompt = ("You will be given a message from a political debate on the internet, and will analyze the statement. Repeat back the statement to analyze."
                        "Then, analyze the following:\n"
                        "-What does the speaker imply about the situation with their statement?\n"
                        "-What does the speaker think about the situation?\n"
                        "-Are what the speaker implies and what the speaker thinks saying the same thing?\n"
                        "Finally, decide if the speaker is pretending to have a certain attitude toward the conversation."
                       )

   sarc_response = anthropic.messages.create(
       model="claude-3-5-sonnet-20240620",
       system=sarc_system_prompt,
       messages=[
           {"role": "user", "content": sample_input}
       ],
       max_tokens=2000,
       temperature=1.0
   )

   judgeInput += "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n"
   judgeInput += sarc_response.content[0].text.strip()

   reflect_system_prompt = ("You will be given a message from a political debate on the internet and a preliminary analysis on the statement. Summarize the preliminary analysis"
                          "Decide whether statement is sarcastic or not by first analyzing the following:\n"
                          "\nThe Implicature - What is implied in the conversation beyond the literal meaning?"
                          "\nThe Presuppositions - What information in the conversation is taken for granted?"
                          "\nThe intent of the speaker - What do the speaker(s) hope to achieve with their statement and who are the speakers?\n"
                          "\nThe polarity - Does the last sentence have a positive or negative tone?"
                          "\nPretense - Is there pretense in the speaker's attitude?"
                          "\nMeaning- What is the difference between the literal and implied meaning of the statement?"
                          "Reflect on the preliminary analysis and what should change, then decide if the statment is sarcastic.")

   reflection_response = anthropic.messages.create(
       model="claude-3-5-sonnet-20240620",
       system=reflect_system_prompt,
       messages=[
           {"role": "user", "content": sample_input + " " + judgeInput}
       ],
       max_tokens=4000,
       temperature=1.0
   )

   judgeInput += "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n"
   judgeInput += reflection_response.content[0].text.strip()
   judgeInput += "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n"
   output += (f"\nANSWER: {judgeInput}\n")

   # Clean output prompt to summarize the decision as YES/NO
   clean_system_prompt = (
       "You will be given the output of an LLM which decided if a sentence is sarcastic or not. "
       "Read the output, then summarize the LLM's stance with ONLY a YES (they think the sentence is sarcastic) or NO (they think the sentence is not sarcastic)."
   )

   clean_output = anthropic.messages.create(
       model="claude-3-5-sonnet-20240620",
       system=clean_system_prompt,
       messages=[
           {"role": "user", "content": judgeInput}
       ],
       max_tokens=10,
       temperature=1.0
   )

   clean_output = clean_output.content[0].text.strip()
   output += (f"Judgement: {clean_output}\n")

   # Store output and determine if the statement is sarcastic
   outputs.append(output)  # Make sure 'outputs' is defined in your scope
   isSarcastic = True if "yes" in clean_output.lower() else False
   return isSarcastic

In [77]:
import random
import time

def beginPrompting(inputs, answers, with_prompt):#, indices):
    num_samples = len(indices)

    with open("SarcasmCorpus_mistakes_V1.txt" ,"a") as file:
            file.write (f"\nIncorrect Answers: (Full output listed below) \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")

    # indices = random.sample(range(len(inputs)), num_samples)
    # sampled_inputs = [inputs[i] for i in indices]
    # sampled_answers = [answers[i] for i in indices]

    isSarCt = 0
    noSarCt = 0
    totNoSarCt = 0
    totSarCt = 0

    # Changed all of the sampled_inputs to inputs and sampled_answers to answers
    
    for x in range(num_samples):
        time.sleep(5)
        isSarcastic = getResultWPrompt(inputs[x]) if with_prompt else getResultNoPrompt(inputs[x])
        if answers[x] == 1:
            totSarCt += 1
            if(isSarcastic):
                isSarCt += 1
            else:
                with open("SarcasmCorpus_mistakes_V1.txt" ,"a") as file:
                    file.write (f"\n{outputs[x]} \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")
                    file.write (f"\nANSWER: {answers[x]}\n")
        else:
            totNoSarCt += 1
            if(not isSarcastic):
                noSarCt += 1
            else:
                with open("SarcasmCorpus_mistakes_V1.txt" ,"a") as file:
                    file.write (f"\n{outputs[x]} \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")
                    file.write (f"\nANSWER: {sampled_answers[x]}\n")
        if with_prompt:
            outputs[x]+=(f"\nANSWER: {sampled_answers[x]}\n")
            outputs[x]+=(f"Prompt sarcastic statement recognition ({isSarCt} total)\n")
            outputs[x]+=(f"Total Prompted sarcastic statement  ({totSarCt} total)  \n")
            outputs[x]+=(f"Non sarcastic statement recognition ({noSarCt} total)\n")
            outputs[x]+=(f"Total Prompted non sarcastic statement ({totNoSarCt} total)\n")
    sar_percentage = (isSarCt / totSarCt * 100) if totSarCt > 0 else 0
    no_sar_percentage = (noSarCt / totNoSarCt * 100) if totNoSarCt > 0 else 0

    return sar_percentage, no_sar_percentage, totSarCt, totNoSarCt, noSarCt, isSarCt


In [79]:
sar_percentage_prompted, no_sar_percentage_prompted, total_sar, total_no_sar, noSarCT, isSarCT = beginPrompting(statements,answers, True, indices)


In [86]:
from datetime import datetime

now = datetime.now()

# Format the date and time as a string
formatted_datetime = now.strftime("%m-%d-%Y %H:%M:%S")

In [87]:
with open("SemEval2018_V1.txt", "a") as file:
    file.write(f"\nSarcasm Corpus V1 ({formatted_datetime}, Total Sample Size : {sampleSize}): \n")
    file.write(f"Prompted sarcastic statement recognition ({total_sar} total) : {sar_percentage_prompted}% \n")
    file.write(f"Total correct: {isSarCT}\n")
    file.write(f"Prompted non sarcastic statement recognition ({total_no_sar} total) : {no_sar_percentage_prompted}%\n")
    file.write(f"Total correct: {noSarCT}\n")
    # file.write(f"No prompt sarcastic statement recognition ({total_sar_no_prompt} total) : {sar_percentage_base}%\n")
    # file.write(f"Total correct: {zeroIsSarCT}")
    # file.write(f"No prompt non sarcastic statement recognition ({total_no_sar_no_prompt} total) : {no_sar_percentage_base}%\n")
    # file.write(f"Total correct: {zeroNoSarCT}")

In [88]:
with open("SemEval2018_results_V1.txt" ,"a") as file:
    for x in range(len(outputs)) :
        file.write (f"\n{outputs[x]} \n- - - - - - - - - - - - - - - - - - - - - - - - - - - \n")